In [14]:
%matplotlib inline
import pandas
import matplotlib
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

columns = ['hour', 'last_windspd', 'last_no', 'last_no2', 'last_nox', 'last_o3', 'no', 'no2', 'nox', 'o3']

plt.rcParams["figure.figsize"] = (20, 12)
plt.rcParams.update({'font.size': 22})

df = pandas.read_csv('data00_100000_clean.csv')
df = df.drop(['AQS_Code', 'Latitude', 'Longitude', 'epoch', 'winddir', 'time', 'date', 'day'], axis=1)
df['last_windspd'] = df['windspd'].shift(1)
df['last_no'] = df['no'].shift(1)
df['last_no2'] = df['no2'].shift(1)
df['last_nox'] = df['nox'].shift(1)
df['last_o3'] = df['o3'].shift(1)
df = df.drop(df.index[[0]])

scaler = MinMaxScaler()
scaler.fit(df[columns])

df = scaler.transform(df[columns])

# Create sequences
sequence_length = 32
num_inputs = 6
num_outputs = 4


data = []
labels = []

rows = deque(maxlen=sequence_length)

for idx, r in enumerate(df):

    rows.append([r[0], r[1], r[2], r[3], r[4], r[5]])
    
    if len(rows) < sequence_length:
        continue
        
    data.append(rows.copy())
    
    labels.append([r[6], r[7], r[8], r[9]])

data = np.array(data)
data.resize((len(data), sequence_length*num_inputs))
labels = np.array(labels)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.33, random_state=42)

/Users/carroll/anaconda3/envs/pollen/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [18]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(random_state=0, n_estimators=100)
regr.fit(X_train, y_train)
regr.score(X_test, y_test)



0.9793417644856737